In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime as dt

In [90]:
# read in all data

airlines = pd.read_csv('./airlines.csv')
airports = pd.read_csv('./airports.csv', encoding='ISO-8859-1')
events_US = pd.read_csv('./events_US.csv', encoding='ISO-8859-1')
fares = pd.read_csv('./fares.csv')
flight_traffic = pd.read_csv('./flight_traffic.csv')

In [15]:
# table of events with the city AND airports that are in it

airports_events_US = airports.merge(events_US, on=['city', 'state'])
airports_events_US['citystate'] = airports_events_US['city'] + '/' + airports_events_US['state']

In [85]:
# table of flight traffic, with cities for the origin and destination airports

airports_flight_traffic = airports.merge(flight_traffic, left_on='airport_id', right_on='origin_airport')
airports_flight_traffic.drop(labels=['airport_id', 'latitude', 'longitude'], axis=1, inplace=True)
airports_flight_traffic.rename(columns={
    'airport_name': 'origin_airport_name',
    'city': 'origin_city',
    'state': 'origin_state'
}, inplace=True)
airports_flight_traffic['origin_citystate'] = airports_flight_traffic['origin_city'] + '/' + airports_flight_traffic['origin_state']
airports_flight_traffic = airports.merge(airports_flight_traffic, left_on='airport_id', right_on='destination_airport')
airports_flight_traffic.drop(labels=['airport_id', 'latitude', 'longitude'], axis=1, inplace=True)
airports_flight_traffic.rename(columns={
    'airport_name': 'destination_airport_name',
    'city': 'destination_city',
    'state': 'destination_state'
}, inplace=True)
airports_flight_traffic['destination_citystate'] = airports_flight_traffic['destination_city'] + '/' + airports_flight_traffic['destination_state']

In [88]:
# lookup table: airport -> city
# not that useful since we have airports_flight_traffic

airport_city = {}
for _, row in airports.iterrows():
    airport_city[row['airport_id']] = row['city'] + '/' + row['state']

In [83]:
# boolean map; True if there is an event in that city and the day queried is within the 10 days preceding an event

citystates = pd.unique(airports_events_US['citystate'])

events_city_date_prev = {}
events_city_date_next = {}
for _, event in airports_events_US.iterrows():
    cs = event['citystate']
    if cs not in events_city_date_prev:
        events_city_date_prev[cs] = set()
    if cs not in events_city_date_next:
        events_city_date_next[cs] = set()
    date_str = event['date']
    dmy = date_str.split("/")
    d = int(mdy[0])
    m = int(mdy[1])
    y = int(mdy[2])
    event_date = dt.datetime(year=y, month=m, day=d)
    preceding_days = set()
    succeeding_days = set()
    for d in range(10):
        preceding_days.add(event_date - dt.timedelta(days=d))
        succeeding_days.add(event_date + dt.timedelta(days=d))
    events_city_date_prev[cs] = preceding_days
    events_city_date_next[cs] = succeeding_days

In [112]:
# PRODUCTION VERSION: USE THIS ONE
# map from city to [incoming_flights_in_prev10_days, outgoing_flights_in_next10_days]
ans = {}
counter = 0
for _, row in airports_flight_traffic.iterrows():
    origin_cs = row['origin_citystate']
    dest_cs = row['destination_citystate']
    if origin_cs not in ans:
        ans[origin_cs] = [0, 0]
    if dest_cs not in ans:
        ans[dest_cs] = [0, 0]
    y, m, d = (row['year'], row['month'], row['day'])
    flight_date = dt.datetime(year=y, month=m, day=d)
    
    # arbitratily use events_city_date_prev to check if theres an event
    if origin_cs in events_city_date_prev and flight_date in events_city_date_next[origin_cs]:
        # i am leaving the origin city in the 10 days after an event
        ans[origin_cs][1] += 1
    if dest_cs in events_city_date_prev and flight_date in events_city_date_prev[dest_cs]:
        # i am arriving in the dest city in the 10 days before an event
        ans[dest_cs][0] += 1
    counter += 1
    if counter % 20000 == 0:
        print(counter)
        
# FOR EACH CITY
# get list of dates of events in that city
# for each of those dates, get number of incoming flights in the 1.5 weeks leading up to that date
# for each of those dates, get number of outgoing flights in the 1.5 weeks after that date
# get the number of incoming flights per 1.5 weeks (average over the whole year)
# get the number of outgoing flights per 1.5 weeks (average over the whole year)
# compare (1-d plot): number of incoming flights preceding event VS over whole year

20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
260000
280000
300000
320000
340000
360000
380000
400000
420000
440000
460000
480000
500000
520000
540000
560000
580000
600000
620000
640000
660000
680000
700000
720000
740000
760000
780000
800000
820000
840000
860000
880000
900000
920000
940000
960000
980000
1000000
1020000
1040000
1060000
1080000
1100000
1120000
1140000
1160000
1180000
1200000
1220000
1240000
1260000
1280000
1300000
1320000
1340000
1360000
1380000
1400000
1420000
1440000
1460000
1480000
1500000
1520000
1540000
1560000
1580000
1600000
1620000
1640000
1660000
1680000
1700000
1720000
1740000
1760000
1780000
1800000
1820000
1840000
1860000
1880000
1900000
1920000
1940000
1960000
1980000
2000000
2020000
2040000
2060000
2080000
2100000
2120000
2140000
2160000
2180000
2200000
2220000
2240000
2260000
2280000
2300000
2320000
2340000
2360000
2380000
2400000
2420000
2440000
2460000
2480000
2500000
2520000
2540000
2560000
2580000
2600000
2620000
26400

In [114]:
ans

{'Aberdeen/SD': [0, 0],
 'Abilene/TX': [0, 0],
 'Adak/AK': [0, 0],
 'Agana/GU': [0, 0],
 'Aguadilla/PR': [0, 0],
 'Akron/OH': [0, 0],
 'Albany/GA': [0, 0],
 'Albany/NY': [0, 0],
 'Albuquerque/NM': [0, 0],
 'Alexandria/LA': [0, 0],
 'Allentown/PA': [0, 0],
 'Alpena/MI': [0, 0],
 'Amarillo/TX': [0, 0],
 'Anchorage/AK': [0, 0],
 'Appleton/WI': [0, 0],
 'Arcata/Eureka/CA': [0, 0],
 'Arlington/VA': [0, 0],
 'Asheville/NC': [0, 0],
 'Aspen/CO': [166, 148],
 'Atlanta/GA': [10443, 10287],
 'Atlantic City/NJ': [100, 100],
 'Augusta/GA': [0, 0],
 'Austin/TX': [1514, 1513],
 'Bakersfield/CA': [0, 0],
 'Baltimore/MD': [2990, 2870],
 'Bangor/ME': [0, 0],
 'Barrow/AK': [0, 0],
 'Baton Rouge/LA': [0, 0],
 'Beaumont/Port Arthur/TX': [0, 0],
 'Bellingham/WA': [0, 0],
 'Bemidji/MN': [0, 0],
 'Bethel/AK': [0, 0],
 'Billings/MT': [0, 0],
 'Binghamton/NY': [0, 0],
 'Birmingham/AL': [0, 0],
 'Bismarck/ND': [0, 0],
 'Bloomington/IL': [0, 0],
 'Boise/ID': [466, 468],
 'Boston/MA': [3604, 3680],
 'Bozeman/MT':

In [127]:
# ans in pandas dataframe format

flight_traffic = pd.DataFrame.from_dict(ans).T.reset_index()
flight_traffic.rename({'index': 'city', 0: 'before', 1: 'after'}, axis=1, inplace=True)
flight_traffic.to_csv('ACTUAL_FULL_FLIGHTS_EVENTS_TRAFFIC.csv', index=False)

flight_active = flight_traffic[(flight_traffic['before'] > 0) | (flight_traffic['after'] > 0)].copy()
flight_active.to_csv('ACTUAL_FLIGHTS_VALID.csv', index=False)

,city,before,after
18,Aspen/CO,166,148
19,Atlanta/GA,10443,10287
20,Atlantic City/NJ,100,100
22,Austin/TX,1514,1513
24,Baltimore/MD,2990,2870
37,Boise/ID,466,468
38,Boston/MA,3604,3680
52,Charleston/SC,430,392
58,Chicago/IL,10939,10943
73,Dallas/TX,1900,1925


In [119]:
# for each city in normalized_ans_counts, calculate average number of flights in 10 days
# total_flights / 365 * 10
# avg_num_flights_10 = map: city -> [avg_num_incoming_flights, avg_num_outgoing_flights] per 10 days

# only perform this operation for the cities that have events in them (i.e. the ones we care about)

# VERY POORLY NAMED: THIS IS THE TOTAL NUM FLIGHTS OVER 365 DAYS
avg_num_flights_10 = {}
for cs, _ in normalized_ans_counts.items():
    avg_num_flights_10[cs] = [0, 0]
    
counter = 0
for _, row in airports_flight_traffic.iterrows():
    origin_cs = row['origin_citystate']
    dest_cs = row['destination_citystate']
    if origin_cs in avg_num_flights_10:
        avg_num_flights_10[origin_cs][1] += 1
    if dest_cs in avg_num_flights_10:
        avg_num_flights_10[dest_cs][0] += 1
    
    counter += 1
    if counter % 20000 == 0:
        print(counter)

20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
260000
280000
300000
320000
340000
360000
380000
400000
420000
440000
460000
480000
500000
520000
540000
560000
580000
600000
620000
640000
660000
680000
700000
720000
740000
760000
780000
800000
820000
840000
860000
880000
900000
920000
940000
960000
980000
1000000
1020000
1040000
1060000
1080000
1100000
1120000
1140000
1160000
1180000
1200000
1220000
1240000
1260000
1280000
1300000
1320000
1340000
1360000
1380000
1400000
1420000
1440000
1460000
1480000
1500000
1520000
1540000
1560000
1580000
1600000
1620000
1640000
1660000
1680000
1700000
1720000
1740000
1760000
1780000
1800000
1820000
1840000
1860000
1880000
1900000
1920000
1940000
1960000
1980000
2000000
2020000
2040000
2060000
2080000
2100000
2120000
2140000
2160000
2180000
2200000
2220000
2240000
2260000
2280000
2300000
2320000
2340000
2360000
2380000
2400000
2420000
2440000
2460000
2480000
2500000
2520000
2540000
2560000
2580000
2600000
2620000
26400

In [122]:
# THIS IS THE RESULT THAT SHOULD HAVE BEEN CALCULATED IN THE PREVIOUS CELL

actual_avg_num_flights_10 = {}
for cs, arr in avg_num_flights_10.items():
    actual_avg_num_flights_10[cs] = [arr[0] / 365 * 10, arr[1] / 365 * 10]
actual_avg_num_flights_10

{'Aspen/CO': [143.4794520547945, 143.5068493150685],
 'Atlanta/GA': [9988.931506849314, 9990.547945205479],
 'Atlantic City/NJ': [96.13698630136986, 96.1917808219178],
 'Austin/TX': [1428.1643835616437, 1427.890410958904],
 'Baltimore/MD': [2767.6438356164385, 2767.5342465753424],
 'Boise/ID': [435.5342465753424, 435.5068493150685],
 'Boston/MA': [3437.041095890411, 3437.150684931507],
 'Charleston/SC': [378.3013698630137, 378.24657534246575],
 'Chicago/IL': [9682.0, 9684.356164383562],
 'Dallas/TX': [1905.205479452055, 1905.0958904109589],
 'Daytona Beach/FL': [55.013698630136986, 55.013698630136986],
 'Detroit/MI': [3471.205479452055, 3468.7397260273974],
 'Eau Claire/WI': [18.52054794520548, 18.52054794520548],
 'Grand Rapids/MI': [298.8493150684932, 298.93150684931504],
 'Harrisburg/PA': [85.01369863013699, 85.01369863013699],
 'Houston/TX': [5031.698630136986, 5032.657534246575],
 'Kansas City/MO': [1234.876712328767, 1234.6575342465753],
 'Key West/FL': [43.397260273972606, 43.36

In [130]:
# average number of flights in 10 days for each [city we care about] to a pandas dataframe
# [city we care about] = cities with events in them

avg_yearly_traffic = pd.DataFrame.from_dict(actual_avg_num_flights_10).T.reset_index()
avg_yearly_traffic.rename({'index': 'city', 0: 'incoming', 1: 'outgoing'}, axis=1, inplace=True)
avg_yearly_traffic.to_csv('AVG_YEARLY_TRAFFIC_PER_10_DAYS.csv', index=False)

In [133]:
# averge the incoming and outgoing flights

avg_in_out_yearly_10_days = (avg_yearly_traffic['incoming'] + avg_yearly_traffic['outgoing']) / 2
avg_in_out_event_days = (flight_active['before'] + flight_active['after']) / 2

In [137]:
flight_traffic.shape

(309, 3)

In [44]:
# this is unrelated to the rest of the notebook
# made this for a teammmate
# puts city names for the origin and destination airports in the "fares" table

airports_fares = airports.merge(fares, left_on='airport_id', right_on='origin_airport')
airports_fares.drop(labels=['airport_id', 'latitude', 'longitude'], axis=1, inplace=True)
airports_fares.rename(columns={
    'airport_name': 'origin_airport_name',
    'city': 'origin_city',
    'state': 'origin_state',
}, inplace=True)
airports_fares['origin_citystate'] = airports_fares['origin_city'] + '/' + airports_fares['origin_state']
airports_fares = airports.merge(airports_fares, left_on='airport_id', right_on='destination_airport')
airports_fares.drop(labels=['airport_id', 'latitude', 'longitude'], axis=1, inplace=True)
airports_fares.rename(columns={
    'airport_name': 'destination_airport_name',
    'city': 'destination_city',
    'state': 'destination_state'
}, inplace=True)
airports_fares['destination_citystate'] = airports_fares['destination_city'] + '/' + airports_fares['destination_state']
airports_fares.to_csv("origin_airports_fares.csv")